In [23]:
from typing import List, Dict, Optional, Literal
from datetime import datetime, timezone
from pydantic import BaseModel, Field, ConfigDict
from crewai import Agent, Task, Crew, Process, LLM
import json
import os


# ============================================
# Data Models
# ============================================
class DeltaOperation(BaseModel):
    """Single mutation to apply to the playbook."""
    model_config = ConfigDict(extra="ignore")
    
    type: Literal["ADD", "UPDATE", "REMOVE"]
    section: str = Field(default="general")
    content: Optional[str] = None
    bullet_id: Optional[str] = None


class DeltaBatch(BaseModel):
    """Bundle of curator reasoning and delta operations."""
    model_config = ConfigDict(extra="ignore")
    
    reasoning: str
    operations: List[DeltaOperation] = Field(default_factory=list)


class Bullet(BaseModel):
    """Single playbook entry."""
    id: str
    section: str
    content: str
    helpful: int = 0
    harmful: int = 0
    neutral: int = 0
    created_at: str = Field(
        default_factory=lambda: datetime.now(timezone.utc).isoformat()
    )
    updated_at: str = Field(
        default_factory=lambda: datetime.now(timezone.utc).isoformat()
    )
    
    def tag(self, tag: Literal["helpful", "harmful", "neutral"], increment: int = 1):
        current = getattr(self, tag)
        setattr(self, tag, current + increment)
        self.updated_at = datetime.now(timezone.utc).isoformat()


class Playbook(BaseModel):
    """Structured context store for self-improving knowledge."""
    bullets: Dict[str, Bullet] = Field(default_factory=dict)
    sections: Dict[str, List[str]] = Field(default_factory=dict)
    next_id: int = 0
    
    def add_bullet(self, section: str, content: str, bullet_id: Optional[str] = None) -> Bullet:
        bullet_id = bullet_id or self._generate_id(section)
        bullet = Bullet(id=bullet_id, section=section, content=content)
        self.bullets[bullet_id] = bullet
        self.sections.setdefault(section, []).append(bullet_id)
        return bullet
    
    def update_bullet(self, bullet_id: str, content: str) -> Optional[Bullet]:
        bullet = self.bullets.get(bullet_id)
        if bullet is None:
            return None
        bullet.content = content
        bullet.updated_at = datetime.now(timezone.utc).isoformat()
        return bullet
    
    def remove_bullet(self, bullet_id: str):
        bullet = self.bullets.pop(bullet_id, None)
        if bullet is None:
            return
        section_list = self.sections.get(bullet.section)
        if section_list:
            self.sections[bullet.section] = [bid for bid in section_list if bid != bullet_id]
            if not self.sections[bullet.section]:
                del self.sections[bullet.section]
    
    def update_bullet_tag(self, bullet_id: str, tag: Literal["helpful", "harmful", "neutral"], 
                        increment: int = 1) -> Optional[Bullet]:
        bullet = self.bullets.get(bullet_id)
        if bullet:
            bullet.tag(tag, increment)
        return bullet
    
    def apply_delta(self, delta: DeltaBatch):
        for operation in delta.operations:
            if operation.type == "ADD":
                self.add_bullet(operation.section, operation.content, operation.bullet_id)
            elif operation.type == "UPDATE" and operation.bullet_id:
                self.update_bullet(operation.bullet_id, operation.content)
            elif operation.type == "REMOVE" and operation.bullet_id:
                self.remove_bullet(operation.bullet_id)
    
    def as_prompt(self) -> str:
        """Return human-readable playbook for prompting."""
        parts = []
        for section, bullet_ids in sorted(self.sections.items()):
            parts.append(f"## {section}")
            for bullet_id in bullet_ids:
                bullet = self.bullets[bullet_id]
                counters = f"(helpful={bullet.helpful}, harmful={bullet.harmful}, neutral={bullet.neutral})"
                parts.append(f"- [{bullet.id}] {bullet.content} {counters}")
        return "\n".join(parts) if parts else "No playbook entries yet."
    
    def _generate_id(self, section: str) -> str:
        self.next_id += 1
        section_prefix = (section or "general").split()[0].lower()
        return f"{section_prefix}-{self.next_id:05d}"
    
    def save(self, path: str):
        with open(path, "w", encoding="utf-8") as f:
            f.write(self.model_dump_json(indent=2))

    @classmethod
    def load(cls, path: str):
        """Load playbook from JSON file, handling missing or empty files."""
        if not os.path.exists(path):
            return cls()
        
        try:
            with open(path, "r", encoding="utf-8") as f:
                content = f.read().strip()
                if not content:  # Handle empty file
                    return cls()
                return cls(**json.loads(content))
        except json.JSONDecodeError as e:
            print(f"Warning: Could not parse {path}. Creating new playbook. Error: {e}")
            return cls()
        except Exception as e:
            print(f"Warning: Error loading {path}. Creating new playbook. Error: {e}")
            return cls()


class GeneratorOutput(BaseModel):
    reasoning: List[str] = Field(description="Step-by-step reasoning process")
    bullet_ids: List[str] = Field(default_factory=list, description="Playbook bullets referenced")
    final_answer: str = Field(description="Concise final answer")


class BulletTag(BaseModel):
    id: str
    tag: Literal["helpful", "harmful", "neutral"]


class Reflection(BaseModel):
    reasoning: str
    error_identification: str
    root_cause_analysis: str
    correct_approach: str
    key_insight: str
    bullet_tags: List[BulletTag] = Field(default_factory=list)


# ============================================
# Agentic RAG Crew
# ============================================
class AgenticCrew:
    def __init__(self, model: str, api_key: str, base_url: str = None,playbook_path: str = "playbook.json"):
        """
        Initialize Agentic RAG Crew.
        
        Args:
            model: Model name (e.g., "gpt-4o-mini", "ollama/llama3.2", "groq/llama-3.1-70b-versatile")
            api_key: API key for the model provider
            base_url: Optional base URL (for Ollama or custom endpoints)
        """
        self.model = model
        self.api_key = api_key
        self.base_url = base_url
        self.playbook_path = playbook_path
        self.playbook = Playbook.load(playbook_path)
        self.context = {}
    
    def create_generator_agent(self) -> Agent:
        """Agent that generates answers using the playbook."""
        return Agent(
            role="Answer Generator",
            goal="Solve problems by referencing the playbook and providing structured reasoning",
            backstory="""You are an expert problem solver who carefully references a curated 
            playbook of strategies and insights. You show your reasoning step-by-step and 
            track which playbook entries you used.""",
            verbose=True,
            allow_delegation=False,
            llm=LLM(model=self.model, api_key=self.api_key, base_url=self.base_url,temperature=0) if self.base_url 
                else LLM(model=self.model, api_key=self.api_key,temperature=0)
        )
    
    def create_reflector_agent(self) -> Agent:
        """Agent that critically analyzes reasoning and identifies improvements."""
        return Agent(
            role="Critical Reflector",
            goal="Analyze reasoning traces to identify errors, patterns, and improvement opportunities",
            backstory="""You are a meticulous analyst who examines problem-solving approaches 
            to find what went wrong and why. You identify root causes and provide actionable 
            insights for improvement.""",
            verbose=True,
            allow_delegation=False,
            llm=LLM(model=self.model, api_key=self.api_key, base_url=self.base_url,temperature=0) if self.base_url 
                else LLM(model=self.model, api_key=self.api_key,temperature=0)
        )
    
    def create_curator_agent(self) -> Agent:
        """Agent that curates and updates the playbook."""
        return Agent(
            role="Playbook Curator",
            goal="Maintain a high-quality playbook by adding new insights and removing outdated content",
            backstory="""You are an expert curator who distills reflections into actionable 
            playbook entries. You avoid duplication, focus on quality over quantity, and 
            ensure the playbook remains organized and useful.""",
            verbose=True,
            allow_delegation=False,
            llm=LLM(model=self.model, api_key=self.api_key, base_url=self.base_url,temperature=0) if self.base_url 
                else LLM(model=self.model, api_key=self.api_key,temperature=0)
        )
    
    def create_generator_task(self, agent: Agent, user_query: str) -> Task:
        """Task for generating an answer."""
        playbook_text = self.playbook.as_prompt()
        
        return Task(
            description=f"""Your task is to answer user queries while providing structured step-by-step reasoning and the bullet IDs you used.
                        input:
                            -User Query: {user_query}
                            -Current Playbook:{playbook_text}

【Required Guidelines】

1. Carefully read the playbook and apply relevant strategies, formulas, and insights
- Check all bullet points in the playbook
- Understand the context and application conditions of each strategy

2. Carefully examine common failures (anti-patterns) listed in the playbook and avoid them
- Present specific alternatives or best practices

3. Show the reasoning process step by step
- Clearly indicate which bullets you referenced at each stage
- Structure so that the logic flow is clear

4. Create thorough but concise analysis
- Include only essential information, but include all central evidence
- Avoid unnecessary repetition

5. Review calculations and logic before providing the final answer
- Confirm that all referenced bullet_ids were actually used
- Check for logical contradictions
- Double-check that you haven't missed any relevant playbook bullets

【Output Rules】
- reasoning: Step-by-step thought process (step-by-step chain of thought), detailed analysis and calculations
- bullet_ids: List of referenced playbook bullet IDs
- final_answer: Clear and verified final answer
""",
            agent=agent,
            expected_output="JSON with reasoning steps, bullet IDs used, and final answer"
        )
    
    def create_reflector_task(self, agent: Agent, user_query: str, generator_output: str) -> Task:
        """Task for reflecting on the generation."""
        playbook_text = self.playbook.as_prompt()
        
        return Task(
            description=f"""Your task is to carefully examine the generator's output, critically analyze it, and create a reflection (JSON).
                        input:
                            -User Query: {user_query}
                            -Generator Output: {generator_output}
                            -Playbook Context:{playbook_text}


【Required Analysis Steps】

1. Carefully analyze the model's reasoning trace to understand where errors occurred
- Review the generator's entire reasoning
- Check for leaps or contradictions in the logic flow

2. Identify specific error types: conceptual errors, calculation mistakes, strategy misuse, etc.
- Clearly describe the characteristics of each error
- Find the root causes behind surface-level errors

3. Provide actionable insights so the model doesn't make the same mistakes in the future
- Present specific procedures or checklists
- Derive generalizable principles

4. Evaluate each bullet point used by the generator
- Tag each bullet_id as ['helpful', 'harmful', 'neutral']
- helpful: bullets that helped with the correct answer
- harmful: incorrect or misleading bullets that led to wrong answers
- neutral: bullets that didn't affect the final result

【Output Rules】
- reasoning: Thought process that went through all 4 analysis steps above, detailed analysis and evidence
- error_identification: Specifically describe what exactly was wrong in the reasoning
- root_cause_analysis: What was the root cause of this error? Which concepts were misunderstood? Which strategies were misused?
- correct_approach: What should the generator have done instead? Present accurate steps and logic
- key_insight: Strategy, formula, principle, or checklist that should be remembered to avoid such errors
- bullet_tags: Tagging results for each bullet referenced by the generator (including id and 'helpful'/'harmful'/'neutral')""",
            agent=agent,
            expected_output="JSON with detailed reflection and bullet tags"
        )
    
    def create_curator_task(self, agent: Agent, user_query: str, reflection: str) -> Task:
        """Task for curating the playbook."""
        playbook_text = self.playbook.as_prompt()
        
        return Task(
            description=f"""You are an expert in curating playbooks.

Considering the existing playbook and reflections from previous attempts:
- Identify only new insights, strategies, and failures that are **missing** from the current playbook
- You can **improve existing bullets with better content** or **remove erroneous/duplicate items**
- Avoid duplication - if similar advice already exists, add only new content that perfectly complements the existing playbook
- Do not regenerate the entire playbook - provide only necessary additions/modifications/deletions
- Focus on quality over quantity - a focused and organized playbook is better than a comprehensive one
- Each change must be specific and justified

User Query: {user_query}

Reflection: {reflection}

Current Playbook:
{playbook_text}


CRITICAL: You must respond with ONLY valid JSON. No markdown, no explanations, no code blocks.

Response format (maximum 3 operations per response):
{{
"reasoning": "Brief explanation (max 200 characters)",
"operations": [
    {{
    "type": "ADD",
    "section": "general",
    "content": "Specific actionable advice (max 150 characters)"
    }},
    {{
    "type": "UPDATE", 
    "bullet_id": "existing-id",
    "content": "Improved content (max 150 characters)"
    }},
    {{
    "type": "REMOVE",
    "bullet_id": "id-to-remove"
    }}
]
}}

Rules:
- Maximum 3 operations per response
- Keep content concise and actionable
- Ensure all JSON strings are properly escaped
- If no changes needed, return: {{"reasoning": "No changes needed", "operations": []}}""",

            agent=agent,
            expected_output="JSON with reasoning and operations (ADD/UPDATE/REMOVE)"
        )
    
    
    def extract_json(self, text: str) -> dict:
        """Extract JSON from text that may contain markdown, think tags, or other noise."""
        text = str(text)
        
        # Remove <think> tags and their content
        import re
        text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
        
        # Remove markdown code blocks
        text = re.sub(r'```json\s*', '', text)
        text = re.sub(r'```\s*', '', text)
        
        # Try to find JSON object
        json_match = re.search(r'\{.*\}', text, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except:
                pass
        
        # If all else fails, return empty structure
        return {}
    def run_ace_cycle(self, user_query: str) -> Dict:
        """Execute one complete ACE (Agent-Curator-Evaluator) cycle."""
        
        # Create agents
        generator = self.create_generator_agent()
        reflector = self.create_reflector_agent()
        curator = self.create_curator_agent()
        
        # Create and execute generator task
        gen_task = self.create_generator_task(generator, user_query)
        gen_crew = Crew(agents=[generator], tasks=[gen_task], process=Process.sequential)
        gen_result = gen_crew.kickoff()
        
        try:
            gen_output = self.extract_json(str(gen_result))
        except:
            gen_output = {"reasoning": [str(gen_result)], "bullet_ids": [], "final_answer": str(gen_result)}
        
        # Create and execute reflector task
        ref_task = self.create_reflector_task(reflector, user_query, json.dumps(gen_output))
        ref_crew = Crew(agents=[reflector], tasks=[ref_task], process=Process.sequential)
        ref_result = ref_crew.kickoff()
        
        try:
            ref_output = self.extract_json(str(ref_result))
            # Update bullet tags in playbook
            for bullet_tag in ref_output.get("bullet_tags", []):
                self.playbook.update_bullet_tag(bullet_tag["id"], bullet_tag["tag"])
        except:
            ref_output = {"reasoning": str(ref_result), "bullet_tags": []}
        
        # Create and execute curator task
        cur_task = self.create_curator_task(curator, user_query, json.dumps(ref_output))
        cur_crew = Crew(agents=[curator], tasks=[cur_task], process=Process.sequential)
        cur_result = cur_crew.kickoff()
        
        try:
            cur_output = self.extract_json(str(cur_result))
            # Apply playbook updates
            delta_batch = DeltaBatch(**cur_output)
            self.playbook.apply_delta(delta_batch)
        except Exception as e:
            print(f"Curator update failed: {e}")
            cur_output = {"reasoning": str(cur_result), "operations": []}
        
        self.playbook.save(self.playbook_path)
        
        return {
            "user_query": user_query,
            "generator_output": gen_output,
            "reflector_output": ref_output,
            "curator_output": cur_output,
            "playbook_stats": {
                "sections": len(self.playbook.sections),
                "bullets": len(self.playbook.bullets)
            }
        }


# ============================================
# Example Usage
# ============================================
if __name__ == "__main__":
        # Example 3: Using Groq
    ace_crew = AgenticCrew(
        model="groq/qwen/qwen3-32b",
        api_key=os.getenv("GROQ_API")
    )
    
    # Run multiple cycles to build up the playbook
    # queries =["A company is designing a recommendation system for an e-commerce website. Should they prioritize accuracy of recommendations or speed/latency for user interactions? Explain the trade-offs and when each priority makes sense."]
    # print("=" * 80)
    # print("AGENTIC RAG WITH SELF-IMPROVING PLAYBOOK")
    # print("=" * 80)
    
    # for i, query in enumerate(queries, 2):
    #     print(f"\n\n{'='*80}")
    #     print(f"CYCLE {i}: {query}")
    #     print(f"{'='*80}\n")

    query="I want to meditate at night, but I still have trouble falling asleep. How should I adjust my practice?"
    result = ace_crew.run_ace_cycle(query)
    
    
    print(f"\n📊 FINAL ANSWER: {result['generator_output'].get('final_answer', 'N/A')}")
    print(f"\n📚 PLAYBOOK STATS: {result['playbook_stats']}")
    print(f"\n📝 CURATOR REASONING: {result['curator_output'].get('reasoning', 'N/A')}")
    
    print("\n\n" + "="*80)
    print("FINAL PLAYBOOK")
    print("="*80)
    print(ace_crew.playbook.as_prompt())

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Answer Generator                                                                                        │
│                                                                                                                 │
│  Task: Your task is to answer user queries while providing structured step-by-step reasoning and the bullet     │
│  IDs you used.                                                                                                  │
│                          input:                                                                                 │
│                              -User Query: I want to meditate at night, but I still have trouble falling         │
│  asleep. How should I adjust my practice?                                                                       │
│                              -Current Playbook:## general                                                       │
│  - [general-00001] Differentiate meditation types: Dynamic (Vipassana) may increase alertness; breath-focused   │
│  (4-7-8) promotes calm (helpful=0, harmful=0, neutral=0)                                                        │
│  - [general-00002] Reduce session duration to 10-15 minutes to avoid cognitive overstimulation before bed       │
│  (helpful=0, harmful=0, neutral=0)                                                                              │
│  - [general-00003] Implement sleep hygiene: Avoid screens 1 hour before bed and track both meditation/sleep     │
│  patterns (helpful=0, harmful=0, neutral=0)                                                                     │
│                                                                                                                 │
│  【Required Guidelines】                                                                                        │
│                                                                                                                 │
│  1. Carefully read the playbook and apply relevant strategies, formulas, and insights                           │
│  - Check all bullet points in the playbook                                                                      │
│  - Understand the context and application conditions of each strategy                                           │
│                                                                                                                 │
│  2. Carefully examine common failures (anti-patterns) listed in the playbook and avoid them                     │
│  - Present specific alternatives or best practices                                                              │
│                                                                                                                 │
│  3. Show the reasoning process step by step                                                                     │
│  - Clearly indicate which bullets you referenced at each stage                                                  │
│  - Structure so that the logic flow is clear                                                                    │
│                                                                                                                 │
│  4. Create thorough but concise analysis                                                                        │
│  - Include only essential information, but include all central evidence                                         │
│  - Avoid unnecessary repetition                                                                                 │
│                                                                                                                 │
│  5. Review calculations and logic before providing the fi

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Answer Generator                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  <think>                                                                                                        │
│  Okay, let's tackle this user's problem. They want to meditate at night but still have trouble falling asleep.  │
│  How should they adjust their practice?                                                                         │
│                                                                                                                 │
│  First, I need to look at the playbook provided. Let me check each bullet point.                                │
│                                                                                                                 │
│  Starting with general-00001: Differentiating meditation types. The user might be using a dynamic meditation    │
│  like Vipassana, which could increase alertness. If they're doing something more active, that might be why      │
│  they can't sleep. So switching to a breath-focused technique like the 4-7-8 method could help promote          │
│  calmness. That makes sense because the 4-7-8 is designed to slow the breath and induce relaxation, which is    │
│  good for sleep.                                                                                                │
│                                                                                                                 │
│  Next, general-00002: Reducing session duration. If they're meditating for too long, maybe 20-30 minutes, that  │
│  could overstimulate their mind before bed. Cutting it down to 10-15 minutes might prevent that cognitive       │
│  overstimulation. Shorter sessions are less likely to keep the brain active, making it easier to transition to  │
│  sleep.                                                                                                         │
│                                                                                                                 │
│  Then, general-00003: Sleep hygiene. They should avoid screens an hour before bed, which is common advice.      │
│  Also, tracking both meditation and sleep patterns can help identify what's working. If they track their        │
│  sessions and sleep, they can see correlations between the type, duration, and timing of meditation and their   │
│  sleep quality. This data can guide further adjustments.                                                        │
│                                                                                                                 │
│  Now, considering common failures. Maybe the user is meditating too close to bedtime, but the playbook doesn't  │
│  mention timing. However, the existing bullets cover type, duration, and hygiene. I should stick to those.      │
│  Also, ensuring they're not using a meditation that's too stimulating is key. The 4-7-8 technique is            │
│  specifically for relaxation, so that's a good alternative.                                                     │
│                                                                                                                 │
│  Putting it all together: The user should switch to a breath-focused meditation, keep sessions short, avoid     │
│  screens, and track their habits. These steps address the alertness from dynamic meditation, prevent            │
│  overstimulation, and improve overall sleep hygiene. Ea

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Critical Reflector                                                                                      │
│                                                                                                                 │
│  Task: Your task is to carefully examine the generator's output, critically analyze it, and create a            │
│  reflection (JSON).                                                                                             │
│                          input:                                                                                 │
│                              -User Query: I want to meditate at night, but I still have trouble falling         │
│  asleep. How should I adjust my practice?                                                                       │
│                              -Generator Output: {"reasoning": ["1. Identify meditation type impact: The user    │
│  may be using a dynamic meditation style (e.g., Vipassana) that increases alertness. Switching to               │
│  breath-focused techniques like 4-7-8 breathing (general-00001) promotes relaxation by slowing respiration and  │
│  calming the nervous system.", "2. Adjust session duration: Limiting meditation to 10-15 minutes                │
│  (general-00002) prevents cognitive overstimulation before bed, which can occur with longer sessions that keep  │
│  the mind active.", "3. Implement sleep hygiene: Avoid screens 1 hour before bed (general-00003) to reduce      │
│  blue light exposure, which disrupts melatonin production. Tracking both meditation and sleep patterns helps    │
│  identify correlations between practice adjustments and sleep quality.", "4. Combine strategies: Using 4-7-8    │
│  breathing (general-00001) for 10-15 minutes (general-00002) followed by a screen-free hour (general-00003)     │
│  creates a cohesive pre-sleep routine that addresses both mental and environmental factors affecting sleep."],  │
│  "bullet_ids": ["general-00001", "general-00002", "general-00003"], "final_answer": "To address sleep           │
│  difficulties while meditating at night: (1) Switch to breath-focused techniques like 4-7-8 breathing to        │
│  reduce alertness, (2) Limit sessions to 10-15 minutes to avoid overstimulation, and (3) Avoid screens 1 hour   │
│  before bed while tracking meditation/sleep patterns to identify optimal adjustments. This combination          │
│  addresses both physiological relaxation and environmental sleep hygiene."}                                     │
│                              -Playbook Context:## general                                                       │
│  - [general-00001] Differentiate meditation types: Dynamic (Vipassana) may increase alertness; breath-focused   │
│  (4-7-8) promotes calm (helpful=0, harmful=0, neutral=0)                                                        │
│  - [general-00002] Reduce session duration to 10-15 minutes to avoid cognitive overstimulation before bed       │
│  (helpful=0, harmful=0, neutral=0)                                                                              │
│  - [general-00003] Implement sleep hygiene: Avoid screens 1 hour before bed and track both meditation/sleep     │
│  patterns (helpful=0, harmful=0, neutral=0)                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  【Required Analysis Steps】                                                                                    │
│                                                          

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Critical Reflector                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  <think>                                                                                                        │
│  Okay, let's start by carefully analyzing the generator's reasoning. The user is having trouble falling asleep  │
│  after meditating at night. The generator's response includes four points: switching to breath-focused          │
│  techniques, adjusting session duration, implementing sleep hygiene, and combining strategies.                  │
│                                                                                                                 │
│  First, I need to check for errors in the reasoning. The first point mentions dynamic meditation like           │
│  Vipassana increasing alertness. That makes sense because dynamic meditations can be more stimulating.          │
│  Switching to 4-7-8 breathing is a good suggestion for relaxation. The second point about limiting sessions to  │
│  10-15 minutes is logical to prevent overstimulation. The third point about avoiding screens and tracking       │
│  patterns is solid sleep hygiene advice. The fourth point combines the strategies, which is a good approach.    │
│                                                                                                                 │
│  Now, looking for any leaps or contradictions. The generator correctly links meditation types to alertness      │
│  levels and suggests specific techniques. The duration adjustment is appropriate. The sleep hygiene advice is   │
│  standard and helpful. The combination of strategies is effective.                                              │
│                                                                                                                 │
│  Next, evaluating each bullet point. The first bullet (general-00001) correctly differentiates meditation       │
│  types, which is helpful. The second (general-00002) about duration is also helpful. The third (general-00003)  │
│  on sleep hygiene is helpful as well. There don't seem to be any harmful or neutral points here. All the        │
│  bullets are directly relevant and contribute positively to the answer.                                         │
│                                                                                                                 │
│  The generator's reasoning is sound. They addressed both the meditation practice and environmental factors      │
│  affecting sleep. There's no obvious error in the logic flow. The strategies are well-supported by the          │
│  playbook context. The final answer effectively combines the recommendations into a cohesive routine.           │
│                                                                                                                 │
│  I don't see any conceptual errors or strategy misuse. The generator correctly applied the playbook elements.   │
│  The root cause analysis would find no issues here. The correct approach was followed, and the key insight is   │
│  integrating both meditation adjustments and sleep hygiene practices. All bullets are helpful.                  │
│  </think>                                                                                                       │
│                                                                                                                 │
│  {                                                     

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Playbook Curator                                                                                        │
│                                                                                                                 │
│  Task: You are an expert in curating playbooks.                                                                 │
│                                                                                                                 │
│  Considering the existing playbook and reflections from previous attempts:                                      │
│  - Identify only new insights, strategies, and failures that are **missing** from the current playbook          │
│  - You can **improve existing bullets with better content** or **remove erroneous/duplicate items**             │
│  - Avoid duplication - if similar advice already exists, add only new content that perfectly complements the    │
│  existing playbook                                                                                              │
│  - Do not regenerate the entire playbook - provide only necessary additions/modifications/deletions             │
│  - Focus on quality over quantity - a focused and organized playbook is better than a comprehensive one         │
│  - Each change must be specific and justified                                                                   │
│                                                                                                                 │
│  User Query: I want to meditate at night, but I still have trouble falling asleep. How should I adjust my       │
│  practice?                                                                                                      │
│                                                                                                                 │
│  Reflection: {"reasoning": "The generator's reasoning is logically structured and addresses both meditation     │
│  practices and sleep hygiene. It correctly identifies dynamic meditation types as potentially stimulating and   │
│  recommends breath-focused techniques. The session duration adjustment and screen avoidance align with sleep    │
│  hygiene principles. The combination of strategies is coherent. No contradictions or leaps in logic are         │
│  present.", "error_identification": "No errors were identified in the reasoning. The generator accurately       │
│  applied the playbook context to address the user's problem.", "root_cause_analysis": "The generator            │
│  successfully integrated meditation type, duration, and environmental factors without conceptual or strategic   │
│  missteps. The advice is evidence-based and contextually appropriate.", "correct_approach": "The generator      │
│  correctly: (1) Differentiated meditation types based on physiological impact, (2) Adjusted session duration    │
│  to prevent cognitive overstimulation, (3) Combined sleep hygiene practices with meditation adjustments. This   │
│  approach is optimal for the scenario.", "key_insight": "When addressing sleep issues related to meditation,    │
│  prioritize: (1) Matching meditation techniques to desired physiological states, (2) Time-bound practices to    │
│  avoid overstimulation, (3) Environmental sleep hygiene integration. Use the playbook context to validate       │
│  strategy alignment.", "bullet_tags": [{"id": "general-00001", "tag": "helpful"}, {"id": "general-00002",       │
│  "tag": "helpful"}, {"id": "general-00003", "tag": "helpful"}]}                                                 │
│                                                                                                                 │
│  Current Playbook:                                     

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Playbook Curator                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  <think>                                                                                                        │
│  Okay, let's tackle this. The user wants to meditate at night but has trouble sleeping. The existing playbook   │
│  has three points: differentiate meditation types, reduce session duration, and implement sleep hygiene. The    │
│  reflection says the reasoning is solid, no errors, but maybe there's something missing.                        │
│                                                                                                                 │
│  Hmm, the user is still having trouble falling asleep. The current advice suggests breath-focused techniques    │
│  and shorter sessions. Maybe there's a need to address the timing of meditation. Like, maybe meditating too     │
│  close to bedtime isn't ideal? Or perhaps incorporating a body scan to relax muscles? Also, the existing        │
│  points mention avoiding screens, but maybe there's more about the environment, like temperature or noise?      │
│  Wait, the reflection's key insight mentions integrating sleep hygiene with meditation. The current playbook    │
│  has track meditation and sleep patterns, but maybe adding a specific technique like progressive muscle         │
│  relaxation could help. Let me check the bullet points again. The existing ones are about meditation type,      │
│  duration, and sleep hygiene. So adding a new technique that's specifically for bedtime, like body scan or      │
│  progressive muscle relaxation, might be a good addition. That would complement the existing advice on          │
│  meditation types. Also, maybe adjusting the timing—suggesting meditation 30-60 minutes before bed instead of   │
│  right before. But the current playbook doesn't mention timing relative to bedtime. So adding a new bullet      │
│  about timing could be helpful. Let me make sure there's no duplication. The existing points don't cover        │
│  timing, so that's a new insight. Alternatively, maybe the session duration is already covered (10-15           │
│  minutes), but timing (when to meditate before bed) isn't. So adding a new point about meditating 30-60         │
│  minutes before bed to allow the body to wind down. That could be an ADD operation. Also, maybe the existing    │
│  bullet about sleep hygiene could be updated to include more environmental factors, but the current one         │
│  mentions avoiding screens and tracking. Alternatively, adding a new bullet for body scan meditation. Let me    │
│  check the character limits. The ADD content needs to be under 150 characters. So something like "Meditate      │
│  30-60 minutes before bed to allow relaxation to set in." Or "Incorporate body scan meditation for physical     │
│  relaxation." Both are concise. Which is more critical? The timing might be more important because meditating   │
│  too close to bed might not let the relaxation take effect. So adding that as a new bullet. Also, maybe the     │
│  existing bullet about session duration (general-00002) could be updated to specify that shorter sessions       │
│  earlier in the evening are better, but the current advice is about duration, not timing. So perhaps adding a   │
│  new bullet for timing. That's a new insight not covered yet. So the operations would be an ADD for timing and  │
│  maybe an UPDATE to the duration bullet to clarify. Wai


📊 FINAL ANSWER: To address sleep difficulties while meditating at night: (1) Switch to breath-focused techniques like 4-7-8 breathing to reduce alertness, (2) Limit sessions to 10-15 minutes to avoid overstimulation, and (3) Avoid screens 1 hour before bed while tracking meditation/sleep patterns to identify optimal adjustments. This combination addresses both physiological relaxation and environmental sleep hygiene.

📚 PLAYBOOK STATS: {'sections': 1, 'bullets': 5}

📝 CURATOR REASONING: Adds timing adjustment and body scan technique to address residual sleep issues not covered by existing entries


FINAL PLAYBOOK
## general
- [general-00001] Differentiate meditation types: Dynamic (Vipassana) may increase alertness; breath-focused (4-7-8) promotes calm (helpful=1, harmful=0, neutral=0)
- [general-00002] Reduce session duration to 10-15 minutes to avoid cognitive overstimulation before bed (helpful=1, harmful=0, neutral=0)
- [general-00003] Implement sleep hygiene: Avoid screens 1 hour